# मेटा फॅमिली मॉडेल्ससह बांधणी

## परिचय

या धड्यात आपण पाहणार आहोत:

- मेटा फॅमिलीमधील दोन मुख्य मॉडेल्स - Llama 3.1 आणि Llama 3.2 यांचा अभ्यास
- प्रत्येक मॉडेलसाठी वापराचे प्रकार आणि परिस्थिती समजून घेणे
- प्रत्येक मॉडेलच्या खास वैशिष्ट्ये दाखवणारा कोड नमुना

## मेटा फॅमिलीचे मॉडेल्स

या धड्यात आपण मेटा फॅमिली किंवा "Llama Herd" मधील २ मॉडेल्स - Llama 3.1 आणि Llama 3.2 यांचा अभ्यास करणार आहोत.

ही मॉडेल्स वेगवेगळ्या प्रकारांमध्ये उपलब्ध आहेत आणि Github Model marketplace वर मिळू शकतात. Github Models वापरून [AI मॉडेल्ससह प्रोटोटायपिंग](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst) कसे करायचे याची अधिक माहिती येथे आहे.

मॉडेल प्रकार:
- Llama 3.1 - 70B Instruct
- Llama 3.1 - 405B Instruct
- Llama 3.2 - 11B Vision Instruct
- Llama 3.2 - 90B Vision Instruct

*टीप: Llama 3 हे देखील Github Models वर उपलब्ध आहे, पण या धड्यात त्याचा समावेश नाही*


## Llama 3.1

४०५ अब्ज पॅरामीटर्ससह, Llama 3.1 हा ओपन सोर्स LLM श्रेणीत येतो.

हा मोड पूर्वीच्या Llama 3 च्या तुलनेत पुढील सुधारणा देतो:

- मोठे कॉन्टेक्स्ट विंडो - १२८k टोकन्स विरुद्ध ८k टोकन्स
- जास्तीत जास्त आउटपुट टोकन्स - ४०९६ विरुद्ध २०४८
- अधिक चांगला बहुभाषिक सपोर्ट - प्रशिक्षण टोकन्समध्ये वाढ झाल्यामुळे

या सुधारणा Llama 3.1 ला GenAI अ‍ॅप्लिकेशन्स तयार करताना अधिक क्लिष्ट वापरासाठी सक्षम करतात, जसे की:
- नेटिव फंक्शन कॉलिंग - LLM वर्कफ्लोच्या बाहेर बाह्य टूल्स आणि फंक्शन्स कॉल करण्याची क्षमता
- अधिक चांगली RAG कार्यक्षमता - मोठ्या कॉन्टेक्स्ट विंडोमुळे
- सिंथेटिक डेटा जनरेशन - फाइन-ट्यूनिंगसारख्या कार्यांसाठी प्रभावी डेटा तयार करण्याची क्षमता


### नेटिव फंक्शन कॉलिंग

Llama 3.1 ला फंक्शन किंवा टूल कॉल्स करण्यासाठी अधिक प्रभावीपणे फाइन-ट्यून करण्यात आले आहे. यामध्ये दोन अंगभूत टूल्स आहेत, ज्या मॉडेलला वापरकर्त्याच्या प्रॉम्प्टनुसार वापरण्याची गरज आहे का हे ओळखता येते. ही टूल्स म्हणजे:

- **Brave Search** - वेब सर्च करून हवामानासारखी ताजी माहिती मिळवण्यासाठी वापरता येते
- **Wolfram Alpha** - अधिक क्लिष्ट गणिती गणनांसाठी वापरता येते, त्यामुळे स्वतःचे फंक्शन्स लिहिण्याची गरज नाही

तुम्ही स्वतःची कस्टम टूल्स देखील तयार करू शकता, जी LLM कॉल करू शकते.

खालील कोड उदाहरणात:

- आपण उपलब्ध टूल्स (brave_search, wolfram_alpha) सिस्टीम प्रॉम्प्टमध्ये परिभाषित करतो.
- वापरकर्त्याचा प्रॉम्प्ट पाठवतो, जो एखाद्या शहरातील हवामानाबद्दल विचारतो.
- LLM Brave Search टूलला कॉल करून प्रतिसाद देईल, जो अशा प्रकारे दिसेल `<|python_tag|>brave_search.call(query="Stockholm weather")`

*टीप: या उदाहरणात फक्त टूल कॉल केले जाते, तुम्हाला प्रत्यक्ष निकाल हवे असतील तर, तुम्हाला Brave API पेजवर मोफत खाते तयार करावे लागेल आणि फंक्शन स्वतः परिभाषित करावे लागेल*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Llama 3.2

LLM असूनही, Llama 3.1 मध्ये एक मर्यादा म्हणजे मल्टिमोडॅलिटी. म्हणजेच, वेगवेगळ्या प्रकारच्या इनपुट्स जसे की प्रतिमा प्रॉम्प्ट म्हणून वापरणे आणि त्यावर उत्तर देणे. ही क्षमता Llama 3.2 ची एक मुख्य वैशिष्ट्य आहे. या वैशिष्ट्यांमध्ये पुढील गोष्टींचा समावेश आहे:

- मल्टिमोडॅलिटी - मजकूर आणि प्रतिमा या दोन्ही प्रकारच्या प्रॉम्प्ट्सचे मूल्यांकन करण्याची क्षमता आहे
- लहान ते मध्यम आकाराचे व्हेरिएंट्स (11B आणि 90B) - हे लवचिक डिप्लॉयमेंट पर्याय उपलब्ध करतात,
- फक्त मजकूर व्हेरिएंट्स (1B आणि 3B) - हे मॉडेलला एज / मोबाइल डिव्हाइसेसवर डिप्लॉय करता येते आणि कमी विलंब प्रदान करते

मल्टिमोडल सपोर्ट हे ओपन सोर्स मॉडेल्सच्या जगात एक मोठे पाऊल आहे. खालील कोड उदाहरणात प्रतिमा आणि मजकूर प्रॉम्प्ट दोन्ही वापरून Llama 3.2 90B कडून प्रतिमेचे विश्लेषण मिळवले आहे.

### Llama 3.2 सह मल्टिमोडल सपोर्ट


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## शिक्षण इथेच थांबत नाही, प्रवास सुरू ठेवा

हे धडा पूर्ण केल्यानंतर, आमच्या [Generative AI Learning collection](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) ला भेट द्या आणि तुमचे Generative AI ज्ञान आणखी वाढवा!



---

**अस्वीकरण**:  
हे दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) वापरून भाषांतरित केले आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात घ्या की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानावा. अत्यावश्यक माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर केल्यामुळे उद्भवणाऱ्या कोणत्याही गैरसमजुती किंवा चुकीच्या अर्थ लावण्यास आम्ही जबाबदार राहणार नाही.
